In [112]:
import pandas as pd
import re
import difflib
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [113]:
df = pd.read_csv("un_country_data_raw.csv")
df.head()

,Country,Category,Indicator,Year,Value
0,Afghanistan,General Information,Region,NaN,Southern Asia
1,Afghanistan,General Information,"Population (000, 2024)",NaN,42 648a
2,Afghanistan,General Information,"Pop. density (per km2, 2024)",NaN,65.7a
3,Afghanistan,General Information,Capital city,NaN,Kabul
4,Afghanistan,General Information,"Capital city pop. (000, 2024)",NaN,4 114.0c


In [114]:
df.head(50)

,Country,Category,Indicator,Year,Value
0,Afghanistan,General Information,Region,NaN,Southern Asia
1,Afghanistan,General Information,"Population (000, 2024)",NaN,42 648a
2,Afghanistan,General Information,"Pop. density (per km2, 2024)",NaN,65.7a
3,Afghanistan,General Information,Capital city,NaN,Kabul
4,Afghanistan,General Information,"Capital city pop. (000, 2024)",NaN,4 114.0c
5,Afghanistan,General Information,UN membership date,NaN,19 November 1946
6,Afghanistan,General Information,Surface area (km2),NaN,652 864b
7,Afghanistan,General Information,Sex ratio (m per 100 f),NaN,102a
8,Afghanistan,General Information,National currency,NaN,Afghani (AFN)
9,Afghanistan,General Information,Exchange rate (per US$),NaN,77.1d


In [115]:
len(df['Country'].unique())

232

## Split the data set by Categories 

In [116]:
df_general_info = df[df["Category"]=="General Information"]
df_economic_indicators = df[df["Category"]=="Economic indicators"]
df_social_indicators = df[df["Category"]=="Social indicators"]
df_env_infra_indicators = df[df["Category"]=="Environment and infrastructure indicators"]
print(len(df_general_info['Country'].unique()))
print(len(df_general_info['Country'].unique()))
print(len(df_general_info['Country'].unique()))
print(len(df_general_info['Country'].unique()))

232
232
232
232


In [117]:
df_general_info = df_general_info.drop(columns=["Category","Year"],axis=1)
df_general_info.head()

,Country,Indicator,Value
0,Afghanistan,Region,Southern Asia
1,Afghanistan,"Population (000, 2024)",42 648a
2,Afghanistan,"Pop. density (per km2, 2024)",65.7a
3,Afghanistan,Capital city,Kabul
4,Afghanistan,"Capital city pop. (000, 2024)",4 114.0c


In [118]:
def pivot_df(df,idx_cols):
    pivot_df = df.pivot(index=idx_cols, columns="Indicator",values="Value")
    flat_df = pivot_df.reset_index()
    flat_df.columns.name = None
    return flat_df

In [119]:
def clean_numerical_cols(df, numerical_cols):
    df_cleaned = df.copy()

    for col in numerical_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].str.replace(r'[^\d.]', '', regex=True)
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        else:
            print(f"Warning: Column '{col}' not found in DataFrame. Skipping.")
            
    return df_cleaned

## General Information

In [120]:
df_general_info_flat = pivot_df(df_general_info, idx_cols=["Country"])
df_general_info_flat.head()

,Country,Capital city,Capital city pop. (000),"Capital city pop. (000, 2024)",Exchange rate (per US$),National currency,"Pop. density (per km2, 2024)","Population (000, 2024)",Region,Sex ratio (m per 100 f),Surface area (km2),UN membership date
0,Afghanistan,Kabul,NaN,4 114.0c,77.1d,Afghani (AFN),65.7a,42 648a,Southern Asia,102a,652 864b,19 November 1946
1,Albania,Tirana,NaN,484.6c,93.9d,Lek (ALL),101.9a,2 792a,Southern Europe,97.8a,28 748b,14 December 1955
2,Algeria,Algiers,NaN,"2 729.3c,d",134.3e,Algerian Dinar (DZD),19.7a,46 814a,Northern Africa,104.1a,2 381 741b,08 October 1962
3,American Samoa,Pago Pago,NaN,48.5d,NaN,US Dollar (USD),"233.8a,b","47a,b",Polynesia,"101.9a,b",199c,NaN
4,Andorra,Andorra la Vella,NaN,22.6c,0.9d,Euro (EUR),174.3a,82a,Southern Europe,104.6a,468b,28 July 1993


### Note

3 Countries has Capital city pop. (000). Assming thy do not have the updated 2024 data

Maybe we can just merge with Capital city pop. (000, 2024)

In [121]:
df_general_info_flat[df_general_info_flat["Capital city pop. (000)"].notna()]["Country"]

111                    Kiribati
154    Northern Mariana Islands
212    Turks and Caicos Islands
Name: Country, dtype: object

In [122]:
numerical_cols = ["Capital city pop. (000)","Capital city pop. (000, 2024)","Exchange rate (per US$)","Pop. density (per km2, 2024)","Population (000, 2024)","Sex ratio (m per 100 f)","Surface area (km2)"]
df_general_info_flat = clean_numerical_cols(df_general_info_flat,numerical_cols)
df_general_info_flat["UN membership date"] = pd.to_datetime(df_general_info_flat["UN membership date"], errors="coerce")


In [123]:
df_general_info_flat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Country                        232 non-null    object        
 1   Capital city                   232 non-null    object        
 2   Capital city pop. (000)        3 non-null      float64       
 3   Capital city pop. (000, 2024)  228 non-null    float64       
 4   Exchange rate (per US$)        216 non-null    float64       
 5   National currency              231 non-null    object        
 6   Pop. density (per km2, 2024)   230 non-null    float64       
 7   Population (000, 2024)         230 non-null    float64       
 8   Region                         232 non-null    object        
 9   Sex ratio (m per 100 f)        230 non-null    float64       
 10  Surface area (km2)             226 non-null    float64       
 11  UN membership date 

In [124]:
df_general_info_flat.head()

,Country,Capital city,Capital city pop. (000),"Capital city pop. (000, 2024)",Exchange rate (per US$),National currency,"Pop. density (per km2, 2024)","Population (000, 2024)",Region,Sex ratio (m per 100 f),Surface area (km2),UN membership date
0,Afghanistan,Kabul,NaN,4114.0,77.1,Afghani (AFN),65.7,42648.0,Southern Asia,102.0,652864.0,1946-11-19
1,Albania,Tirana,NaN,484.6,93.9,Lek (ALL),101.9,2792.0,Southern Europe,97.8,28748.0,1955-12-14
2,Algeria,Algiers,NaN,2729.3,134.3,Algerian Dinar (DZD),19.7,46814.0,Northern Africa,104.1,2381741.0,1962-10-08
3,American Samoa,Pago Pago,NaN,48.5,NaN,US Dollar (USD),233.8,47.0,Polynesia,101.9,199.0,NaT
4,Andorra,Andorra la Vella,NaN,22.6,0.9,Euro (EUR),174.3,82.0,Southern Europe,104.6,468.0,1993-07-28


### Note

Some missing values. Dropping might be the solution? Further analysis on the nulls are needed 

In [125]:
df_general_info_flat.isnull().sum()

Country                            0
Capital city                       0
Capital city pop. (000)          229
Capital city pop. (000, 2024)      4
Exchange rate (per US$)           16
National currency                  1
Pop. density (per km2, 2024)       2
Population (000, 2024)             2
Region                             0
Sex ratio (m per 100 f)            2
Surface area (km2)                 6
UN membership date                39
dtype: int64

## Economic Indicators

In [126]:
df_economic_indicators = df_economic_indicators.drop(columns=["Category"],axis=1)
df_economic_indicators.head()

,Country,Indicator,Year,Value
10,Afghanistan,GDP: Gross domestic product (million current US$),2015.0,18 700
11,Afghanistan,GDP: Gross domestic product (million current US$),2020.0,19 983
12,Afghanistan,GDP: Gross domestic product (million current US$),2024.0,14 175b
13,Afghanistan,"GDP growth rate (annual %, const. 2015 prices)",2015.0,-2.5
14,Afghanistan,"GDP growth rate (annual %, const. 2015 prices)",2020.0,-2.1


In [127]:
df_economic_indicators_flat = pivot_df(df_economic_indicators, idx_cols=["Country","Year"])
df_economic_indicators_flat.head()


,Country,Year,Agricultural production index (2014-2016=100),Agricultural production indexa (2014-2016=100),"Balance of payments, current account (million US$)","Balance of payments, current accounta (million US$)",CPI: Consumer Price Index (2010=100),CPI: Consumer Price Indexe (2010=100),"CPI: Consumer Price Indexe,n (2010=100)",CPI: Consumer Price Indexf (2010=100),"CPI: Consumer Price Indexf,g (2010=100)",CPI: Consumer Price Indexg (2010=100),"CPI: Consumer Price Indexg,h (2010=100)",CPI: Consumer Price Indexh (2010=100),CPI: Consumer Price Indexi (2010=100),"CPI: Consumer Price Indexi,j (2010=100)",CPI: Consumer Price Indexj (2010=100),"CPI: Consumer Price Indexj,k (2010=100)",CPI: Consumer Price Indexk (2010=100),"CPI: Consumer Price Indexk,l (2010=100)",CPI: Consumer Price Indexl (2010=100),"CPI: Consumer Price Indexl,m (2010=100)",CPI: Consumer Price Indexm (2010=100),"CPI: Consumer Price Indexm,n (2010=100)",CPI: Consumer Price Indexn (2010=100),Economy: Agriculture (% of Gross Value Added),Economy: Agricultured (% of Gross Value Added),"Economy: Agricultured,e (% of Gross Value Added)",Economy: Agriculturee (% of Gross Value Added),"Economy: Agriculturee,f (% of Gross Value Added)","Economy: Agriculturee,f,g (% of Gross Value Added)","Economy: Agriculturee,g,h (% of Gross Value Added)",Economy: Agriculturef (% of Gross Value Added),"Economy: Agriculturef,g (% of Gross Value Added)","Economy: Agriculturef,g,h (% of Gross Value Added)",Economy: Agricultureg (% of Gross Value Added),"Economy: Agricultureg,h (% of Gross Value Added)","Economy: Agricultureg,h,i (% of Gross Value Added)","Economy: Agricultureh,i (% of Gross Value Added)","Economy: Agricultureh,i,j (% of Gross Value Added)",Economy: Industry (% of Gross Value Added),Economy: Industryd (% of Gross Value Added),"Economy: Industryd,f (% of Gross Value Added)",Economy: Industrye (% of Gross Value Added),"Economy: Industrye,f (% of Gross Value Added)","Economy: Industrye,g (% of Gross Value Added)","Economy: Industrye,g,i (% of Gross Value Added)","Economy: Industrye,h (% of Gross Value Added)",Economy: Industryf (% of Gross Value Added),"Economy: Industryf,g (% of Gross Value Added)","Economy: Industryf,g,h (% of Gross Value Added)","Economy: Industryf,g,i (% of Gross Value Added)","Economy: Industryf,h (% of Gross Value Added)",Economy: Industryg (% of Gross Value Added),"Economy: Industryg,h (% of Gross Value Added)","Economy: Industryg,h,i (% of Gross Value Added)","Economy: Industryg,i (% of Gross Value Added)","Economy: Industryg,i,j (% of Gross Value Added)","Economy: Industryg,j (% of Gross Value Added)",Economy: Industryh (% of Gross Value Added),"Economy: Industryh,i (% of Gross Value Added)","Economy: Industryh,i,j (% of Gross Value Added)","Economy: Industryh,i,k (% of Gross Value Added)","Economy: Industryh,j (% of Gross Value Added)","Economy: Industryi,j (% of Gross Value Added)",Economy: Services and other activity (% of GVA),Economy: Services and other activityd (% of GVA),"Economy: Services and other activityd,g (% of GVA)",Economy: Services and other activitye (% of GVA),"Economy: Services and other activitye,f (% of GVA)","Economy: Services and other activitye,f,g (% of GVA)","Economy: Services and other activitye,f,i (% of GVA)","Economy: Services and other activitye,g (% of GVA)","Economy: Services and other activitye,g,j (% of GVA)","Economy: Services and other activitye,h (% of GVA)",Economy: Services and other activityf (% of GVA),"Economy: Services and other activityf,g (% of GVA)","Economy: Services and other activityf,g,i (% of GVA)","Economy: Services and other activityf,g,j (% of GVA)","Economy: Services and other activityf,h (% of GVA)","Economy: Services and other activityf,i (% of GVA)",Economy: Services and other activityg (% of GVA),"Economy: Services and other activityg,h (% of GVA)","Economy: Services and other activityg,i (% of GVA)","Economy: Services and other activityg,j (% of GVA)","Economy: Services and other activityg,k 

In [128]:
def smart_combine_typo_columns(df,root_forms):
    """
    Merge typo columns into their root forms using similarity matching
    """
    df_fixed = df.copy()
    
    # For each root form, find and merge similar columns
    for root_col in root_forms:
        if root_col in df_fixed.columns:
            similar_columns = []
            
            # Check all columns for similarity to this root form
            for col in df_fixed.columns:
                if col != root_col and col not in root_forms:  # Don't compare with itself or other root forms
                    similarity = difflib.SequenceMatcher(None, root_col.lower(), col.lower()).ratio()
                    
                    if similarity > 0.8:  # High similarity threshold
                        similar_columns.append((col, similarity))
            
            # Merge similar columns into the root column
            if similar_columns:
                print(f"\nMerging into '{root_col}':")
                print(f"  Found {len(similar_columns)} similar columns")
                
                for similar_col, sim_score in similar_columns:
                    print(f"  Merging: '{similar_col}' -> '{root_col}' (similarity: {sim_score:.3f})")
                    # Fill missing values in root column with values from similar column
                    df_fixed[root_col] = df_fixed[root_col].fillna(df_fixed[similar_col])
                    # Drop the similar column
                    df_fixed = df_fixed.drop(columns=[similar_col])
    
    return df_fixed


In [129]:
print(list(df_economic_indicators_flat.columns))

['Country', 'Year', 'Agricultural production index (2014-2016=100)', 'Agricultural production indexa (2014-2016=100)', 'Balance of payments, current account (million US$)', 'Balance of payments, current accounta (million US$)', 'CPI: Consumer Price Index (2010=100)', 'CPI: Consumer Price Indexe (2010=100)', 'CPI: Consumer Price Indexe,n (2010=100)', 'CPI: Consumer Price Indexf (2010=100)', 'CPI: Consumer Price Indexf,g (2010=100)', 'CPI: Consumer Price Indexg (2010=100)', 'CPI: Consumer Price Indexg,h (2010=100)', 'CPI: Consumer Price Indexh (2010=100)', 'CPI: Consumer Price Indexi (2010=100)', 'CPI: Consumer Price Indexi,j (2010=100)', 'CPI: Consumer Price Indexj (2010=100)', 'CPI: Consumer Price Indexj,k (2010=100)', 'CPI: Consumer Price Indexk (2010=100)', 'CPI: Consumer Price Indexk,l (2010=100)', 'CPI: Consumer Price Indexl (2010=100)', 'CPI: Consumer Price Indexl,m (2010=100)', 'CPI: Consumer Price Indexm (2010=100)', 'CPI: Consumer Price Indexm,n (2010=100)', 'CPI: Consumer Pric

In [130]:
['Country', 'Year', 'Agricultural production index (2014-2016=100)', 'Agricultural production indexa (2014-2016=100)', 'Balance of payments, current account (million US$)', 'Balance of payments, current accounta (million US$)', 'CPI: Consumer Price Index (2010=100)', 'CPI: Consumer Price Indexe (2010=100)', 'CPI: Consumer Price Indexe,n (2010=100)', 'CPI: Consumer Price Indexf (2010=100)', 'CPI: Consumer Price Indexf,g (2010=100)', 'CPI: Consumer Price Indexg (2010=100)', 'CPI: Consumer Price Indexg,h (2010=100)', 'CPI: Consumer Price Indexh (2010=100)', 'CPI: Consumer Price Indexi (2010=100)', 'CPI: Consumer Price Indexi,j (2010=100)', 'CPI: Consumer Price Indexj (2010=100)', 'CPI: Consumer Price Indexj,k (2010=100)', 'CPI: Consumer Price Indexk (2010=100)', 'CPI: Consumer Price Indexk,l (2010=100)', 'CPI: Consumer Price Indexl (2010=100)', 'CPI: Consumer Price Indexl,m (2010=100)', 'CPI: Consumer Price Indexm (2010=100)', 'CPI: Consumer Price Indexm,n (2010=100)', 'CPI: Consumer Price Indexn (2010=100)', 'Economy: Agriculture (% of Gross Value Added)', 'Economy: Agricultured (% of Gross Value Added)', 'Economy: Agricultured,e (% of Gross Value Added)', 'Economy: Agriculturee (% of Gross Value Added)', 'Economy: Agriculturee,f (% of Gross Value Added)', 'Economy: Agriculturee,f,g (% of Gross Value Added)', 'Economy: Agriculturee,g,h (% of Gross Value Added)', 'Economy: Agriculturef (% of Gross Value Added)', 'Economy: Agriculturef,g (% of Gross Value Added)', 'Economy: Agriculturef,g,h (% of Gross Value Added)', 'Economy: Agricultureg (% of Gross Value Added)', 'Economy: Agricultureg,h (% of Gross Value Added)', 'Economy: Agricultureg,h,i (% of Gross Value Added)', 'Economy: Agricultureh,i (% of Gross Value Added)', 'Economy: Agricultureh,i,j (% of Gross Value Added)', 'Economy: Industry (% of Gross Value Added)', 'Economy: Industryd (% of Gross Value Added)', 'Economy: Industryd,f (% of Gross Value Added)', 'Economy: Industrye (% of Gross Value Added)', 'Economy: Industrye,f (% of Gross Value Added)', 'Economy: Industrye,g (% of Gross Value Added)', 'Economy: Industrye,g,i (% of Gross Value Added)', 'Economy: Industrye,h (% of Gross Value Added)', 'Economy: Industryf (% of Gross Value Added)', 'Economy: Industryf,g (% of Gross Value Added)', 'Economy: Industryf,g,h (% of Gross Value Added)', 'Economy: Industryf,g,i (% of Gross Value Added)', 'Economy: Industryf,h (% of Gross Value Added)', 'Economy: Industryg (% of Gross Value Added)', 'Economy: Industryg,h (% of Gross Value Added)', 'Economy: Industryg,h,i (% of Gross Value Added)', 'Economy: Industryg,i (% of Gross Value Added)', 'Economy: Industryg,i,j (% of Gross Value Added)', 'Economy: Industryg,j (% of Gross Value Added)', 'Economy: Industryh (% of Gross Value Added)', 'Economy: Industryh,i (% of Gross Value Added)', 'Economy: Industryh,i,j (% of Gross Value Added)', 'Economy: Industryh,i,k (% of Gross Value Added)', 'Economy: Industryh,j (% of Gross Value Added)', 'Economy: Industryi,j (% of Gross Value Added)', 'Economy: Services and other activity (% of GVA)', 'Economy: Services and other activityd (% of GVA)', 'Economy: Services and other activityd,g (% of GVA)', 'Economy: Services and other activitye (% of GVA)', 'Economy: Services and other activitye,f (% of GVA)', 'Economy: Services and other activitye,f,g (% of GVA)', 'Economy: Services and other activitye,f,i (% of GVA)', 'Economy: Services and other activitye,g (% of GVA)', 'Economy: Services and other activitye,g,j (% of GVA)', 'Economy: Services and other activitye,h (% of GVA)', 'Economy: Services and other activityf (% of GVA)', 'Economy: Services and other activityf,g (% of GVA)', 'Economy: Services and other activityf,g,i (% of GVA)', 'Economy: Services and other activityf,g,j (% of GVA)', 'Economy: Services and other activityf,h (% of GVA)', 'Economy: Services and other activityf,i (% of GVA)', 'Economy: Services and other activityg (% of GVA)', 'Economy: Services and other activityg,h (% of GVA)', 'Economy: Services and other activityg,i (% of GVA)', 'Economy: Services and other activityg,j (% of GVA)', 'Economy: Services and other activityg,k (% of GVA)', 'Economy: Services and other activityh (% of GVA)', 'Economy: Services and other activityh,i,l (% of GVA)', 'Economy: Services and other activityh,j (% of GVA)', 'Economy: Services and other activityh,k (% of GVA)', 'Economy: Services and other activityi (% of GVA)', 'Economy: Services and other activityi,j (% of GVA)', 'Economy: Services and other activityi,k (% of GVA)', 'Economy: Services and other activityj (% of GVA)', 'Economy: Services and other activityk (% of GVA)', 'Employment in agriculture (% of employed)', 'Employment in agriculturea,k (% of employed)', 'Employment in agricultured (% of employed)', 'Employment in agriculturee (% of employed)', 'Employment in agriculturef (% of employed)', 'Employment in agricultureg (% of employed)', 'Employment in agricultureh (% of employed)', 'Employment in agriculturei (% of employed)', 'Employment in agriculturej (% of employed)', 'Employment in agriculturek (% of employed)', 'Employment in agriculturel (% of employed)', 'Employment in agriculturem (% of employed)', 'Employment in industry (% of employed)', 'Employment in industrya,k (% of employed)', 'Employment in industryd (% of employed)', 'Employment in industrye (% of employed)', 'Employment in industryf (% of employed)', 'Employment in industryg (% of employed)', 'Employment in industryh (% of employed)', 'Employment in industryi (% of employed)', 'Employment in industryj (% of employed)', 'Employment in industryk (% of employed)', 'Employment in industryl (% of employed)', 'Employment in industrym (% of employed)', 'Employment in services (% employed)', 'Employment in servicesa,k (% employed)', 'Employment in servicesd (% employed)', 'Employment in servicese (% employed)', 'Employment in servicesf (% employed)', 'Employment in servicesg (% employed)', 'Employment in servicesh (% employed)', 'Employment in servicesi (% employed)', 'Employment in servicesj (% employed)', 'Employment in servicesk (% employed)', 'Employment in servicesl (% employed)', 'Employment in servicesm (% employed)', 'GDP growth rate (annual %, const. 2015 prices)', 'GDP growth ratea (annual %, const. 2015 prices)', 'GDP growth ratee (annual %, const. 2015 prices)', 'GDP growth ratef (annual %, const. 2015 prices)', 'GDP per capita (current US$)', 'GDP per capitaa (current US$)', 'GDP per capitad (current US$)', 'GDP per capitae (current US$)', 'GDP per capitaf (current US$)', 'GDP: Gross domestic product (million current US$)', 'GDP: Gross domestic producta (million current US$)', 'GDP: Gross domestic productd (million current US$)', 'GDP: Gross domestic producte (million current US$)', 'GDP: Gross domestic productf (million current US$)', 'International trade: balance (million current US$)', 'International trade: balancea (million current US$)', 'International trade: balanced (million current US$)', 'International trade: balancee (million current US$)', 'International trade: balancef (million current US$)', 'International trade: balanceg (million current US$)', 'International trade: balanceh (million current US$)', 'International trade: balancei (million current US$)', 'International trade: balancej (million current US$)', 'International trade: balancek (million current US$)', 'International trade: balancel (million current US$)', 'International trade: balancen (million current US$)', 'International trade: balanceo (million current US$)', 'International trade: exports (million current US$)', 'International trade: exportsa (million current US$)', 'International trade: exportsd (million current US$)', 'International trade: exportse (million current US$)', 'International trade: exportsf (million current US$)', 'International trade: exportsg (million current US$)', 'International trade: exportsh (million current US$)', 'International trade: exportsi (million current US$)', 'International trade: exportsj (million current US$)', 'International trade: exportsk (million current US$)', 'International trade: exportsl (million current US$)', 'International trade: exportsn (million current US$)', 'International trade: exportso (million current US$)', 'International trade: imports (million current US$)', 'International trade: importsa (million current US$)', 'International trade: importsd (million current US$)', 'International trade: importse (million current US$)', 'International trade: importsf (million current US$)', 'International trade: importsg (million current US$)', 'International trade: importsh (million current US$)', 'International trade: importsi (million current US$)', 'International trade: importsj (million current US$)', 'International trade: importsk (million current US$)', 'International trade: importsl (million current US$)', 'International trade: importsn (million current US$)', 'International trade: importso (million current US$)', 'Labour force participation rate (female/male pop. %)', 'Labour force participation ratea,k (female/male pop. %)', 'Labour force participation rated (female/male pop. %)', 'Labour force participation ratee (female/male pop. %)', 'Labour force participation ratef (female/male pop. %)', 'Labour force participation rateg (female/male pop. %)', 'Labour force participation rateh (female/male pop. %)', 'Labour force participation ratei (female/male pop. %)', 'Labour force participation ratej (female/male pop. %)', 'Labour force participation ratek (female/male pop. %)', 'Labour force participation ratem (female/male pop. %)', 'Unemployment (% of labour force)', 'Unemploymenta,k (% of labour force)', 'Unemploymentd (% of labour force)', 'Unemploymente (% of labour force)', 'Unemploymentf (% of labour force)', 'Unemploymentg (% of labour force)', 'Unemploymenth (% of labour force)', 'Unemploymenti (% of labour force)', 'Unemploymentj (% of labour force)', 'Unemploymentk (% of labour force)']


['Country',
 'Year',
 'Agricultural production index (2014-2016=100)',
 'Agricultural production indexa (2014-2016=100)',
 'Balance of payments, current account (million US$)',
 'Balance of payments, current accounta (million US$)',
 'CPI: Consumer Price Index (2010=100)',
 'CPI: Consumer Price Indexe (2010=100)',
 'CPI: Consumer Price Indexe,n (2010=100)',
 'CPI: Consumer Price Indexf (2010=100)',
 'CPI: Consumer Price Indexf,g (2010=100)',
 'CPI: Consumer Price Indexg (2010=100)',
 'CPI: Consumer Price Indexg,h (2010=100)',
 'CPI: Consumer Price Indexh (2010=100)',
 'CPI: Consumer Price Indexi (2010=100)',
 'CPI: Consumer Price Indexi,j (2010=100)',
 'CPI: Consumer Price Indexj (2010=100)',
 'CPI: Consumer Price Indexj,k (2010=100)',
 'CPI: Consumer Price Indexk (2010=100)',
 'CPI: Consumer Price Indexk,l (2010=100)',
 'CPI: Consumer Price Indexl (2010=100)',
 'CPI: Consumer Price Indexl,m (2010=100)',
 'CPI: Consumer Price Indexm (2010=100)',
 'CPI: Consumer Price Indexm,n (2010=100

In [131]:
root_forms = [
    'Country',
    'Year',
    'Agricultural production index (2014-2016=100)',
    'Balance of payments, current account (million US$)',
    'CPI: Consumer Price Index (2010=100)',
    'Economy: Agriculture (% of Gross Value Added)',
    'Economy: Industry (% of Gross Value Added)',
    'Economy: Services and other activity (% of GVA)',
    'Employment in agriculture (% of employed)',
    'Employment in industry (% of employed)',
    'Employment in services (% employed)',
    'GDP growth rate (annual %, const. 2015 prices)',
    'GDP per capita (current US$)',
    'GDP: Gross domestic product (million current US$)',
    'International trade: balance (million current US$)',
    'International trade: exports (million current US$)',
    'International trade: imports (million current US$)',
    'Labour force participation rate (female/male pop. %)',
    'Unemployment (% of labour force)'
]

df_economic_indicators_flat = smart_combine_typo_columns(df_economic_indicators_flat, root_forms)
print(f"\nFinal shape: {df_economic_indicators_flat.shape}")
print(f"Final columns: {list(df_economic_indicators_flat.columns)}")
df_economic_indicators_flat.head()


Merging into 'Agricultural production index (2014-2016=100)':
  Found 1 similar columns
  Merging: 'Agricultural production indexa (2014-2016=100)' -> 'Agricultural production index (2014-2016=100)' (similarity: 0.989)

Merging into 'Balance of payments, current account (million US$)':
  Found 1 similar columns
  Merging: 'Balance of payments, current accounta (million US$)' -> 'Balance of payments, current account (million US$)' (similarity: 0.990)

Merging into 'CPI: Consumer Price Index (2010=100)':
  Found 18 similar columns
  Merging: 'CPI: Consumer Price Indexe (2010=100)' -> 'CPI: Consumer Price Index (2010=100)' (similarity: 0.986)
  Merging: 'CPI: Consumer Price Indexe,n (2010=100)' -> 'CPI: Consumer Price Index (2010=100)' (similarity: 0.960)
  Merging: 'CPI: Consumer Price Indexf (2010=100)' -> 'CPI: Consumer Price Index (2010=100)' (similarity: 0.986)
  Merging: 'CPI: Consumer Price Indexf,g (2010=100)' -> 'CPI: Consumer Price Index (2010=100)' (similarity: 0.960)
  Mergin

,Country,Year,Agricultural production index (2014-2016=100),"Balance of payments, current account (million US$)",CPI: Consumer Price Index (2010=100),Economy: Agriculture (% of Gross Value Added),Economy: Industry (% of Gross Value Added),Economy: Services and other activity (% of GVA),Employment in agriculture (% of employed),Employment in industry (% of employed),Employment in services (% employed),"GDP growth rate (annual %, const. 2015 prices)",GDP per capita (current US$),GDP: Gross domestic product (million current US$),International trade: balance (million current US$),International trade: exports (million current US$),International trade: imports (million current US$),Labour force participation rate (female/male pop. %),Unemployment (% of labour force)
0,Afghanistan,2015.0,96,- 4 193,133,22,NaN,68.3,44.6,20.7,NaN,-2.5,554,18 700,- 7 151,571,7 723,19.1 / 75.3i,9i
1,Afghanistan,2020.0,112,- 3 137,150c,31.4f,NaN,55h,46,18.5,NaN,-2.1,512.7,19 983,- 7 484i,783i,8 266i,16.5 / 66.7,11.7
2,Afghanistan,2024.0,111b,...,...,"35.5f,b",NaN,"47.5h,b",46.6b,18.3b,NaN,-6.2b,344.6b,14 175b,"- 4 337i,j","456i,j","4 793i,j",5.0 / 69.3i,16.3i
3,Albania,2015.0,100,-980,111,22.5,24.8,52.7,41.3,18.7,NaN,2.2,3 950.4,11 387,- 2 391,NaN,NaN,47.0 / 64.1,17.2
4,Albania,2020.0,106,- 1 315,121,21.9,22.9,55.1,36.2,20.6,NaN,-3.5,5 289.0,15 163,- 2 995,NaN,NaN,50.1 / 64.7i,12.8i


In [132]:
df_economic_indicators_flat.isnull().sum()

Country                                                   0
Year                                                      0
Agricultural production index (2014-2016=100)            93
Balance of payments, current account (million US$)      102
CPI: Consumer Price Index (2010=100)                     72
Economy: Agriculture (% of Gross Value Added)            66
Economy: Industry (% of Gross Value Added)               75
Economy: Services and other activity (% of GVA)          57
Employment in agriculture (% of employed)                81
Employment in industry (% of employed)                   78
Employment in services (% employed)                     639
GDP growth rate (annual %, const. 2015 prices)           57
GDP per capita (current US$)                             57
GDP: Gross domestic product (million current US$)        57
International trade: balance (million current US$)       54
International trade: exports (million current US$)      195
International trade: imports (million cu

In [133]:
numerical_cols = [
    'Agricultural production index (2014-2016=100)',
    'Balance of payments, current account (million US$)',
    'CPI: Consumer Price Index (2010=100)',
    'Economy: Agriculture (% of Gross Value Added)',
    'Economy: Industry (% of Gross Value Added)',
    'Economy: Services and other activity (% of GVA)',
    'Employment in agriculture (% of employed)',
    'Employment in industry (% of employed)',
    'Employment in services (% employed)',
    'GDP growth rate (annual %, const. 2015 prices)',
    'GDP per capita (current US$)',
    'GDP: Gross domestic product (million current US$)',
    'International trade: balance (million current US$)',
    'International trade: exports (million current US$)',
    'International trade: imports (million current US$)',
    'Unemployment (% of labour force)'
]



In [134]:
df_economic_indicators_flat = clean_numerical_cols(df_economic_indicators_flat, numerical_cols)
df_economic_indicators_flat.head()

,Country,Year,Agricultural production index (2014-2016=100),"Balance of payments, current account (million US$)",CPI: Consumer Price Index (2010=100),Economy: Agriculture (% of Gross Value Added),Economy: Industry (% of Gross Value Added),Economy: Services and other activity (% of GVA),Employment in agriculture (% of employed),Employment in industry (% of employed),Employment in services (% employed),"GDP growth rate (annual %, const. 2015 prices)",GDP per capita (current US$),GDP: Gross domestic product (million current US$),International trade: balance (million current US$),International trade: exports (million current US$),International trade: imports (million current US$),Labour force participation rate (female/male pop. %),Unemployment (% of labour force)
0,Afghanistan,2015.0,96.0,4193.0,133.0,22.0,NaN,68.3,44.6,20.7,NaN,2.5,554.0,18700.0,7151.0,571.0,7723.0,19.1 / 75.3i,9.0
1,Afghanistan,2020.0,112.0,3137.0,150.0,31.4,NaN,55.0,46.0,18.5,NaN,2.1,512.7,19983.0,7484.0,783.0,8266.0,16.5 / 66.7,11.7
2,Afghanistan,2024.0,111.0,NaN,NaN,35.5,NaN,47.5,46.6,18.3,NaN,6.2,344.6,14175.0,4337.0,456.0,4793.0,5.0 / 69.3i,16.3
3,Albania,2015.0,100.0,980.0,111.0,22.5,24.8,52.7,41.3,18.7,NaN,2.2,3950.4,11387.0,2391.0,NaN,NaN,47.0 / 64.1,17.2
4,Albania,2020.0,106.0,1315.0,121.0,21.9,22.9,55.1,36.2,20.6,NaN,3.5,5289.0,15163.0,2995.0,NaN,NaN,50.1 / 64.7i,12.8


In [135]:
def split_ratio_column(df, column_to_split, new_col1_name, new_col2_name):
    """
    Split a column with format "value1 / value2" into separate columns
    
    Parameters:
    df: DataFrame to modify
    column_to_split: Name of the column to split (should contain values like "19.1 / 75.3i")
    new_col1_name: Name for the first value column
    new_col2_name: Name for the second value column
    
    Returns:
    DataFrame with new columns added and ratio calculated
    """
    df_result = df.copy()
    
    if column_to_split in df_result.columns:
        # Split the column on '/'
        split_data = df_result[column_to_split].str.split('/', expand=True)
        
        if split_data.shape[1] >= 2:
            # Clean and convert first value
            df_result[new_col1_name] = split_data[0].str.strip().str.replace(r'[^\d.]', '', regex=True)
            df_result[new_col1_name] = pd.to_numeric(df_result[new_col1_name], errors='coerce')
            
            # Clean and convert second value
            df_result[new_col2_name] = split_data[1].str.strip().str.replace(r'[^\d.]', '', regex=True)
            df_result[new_col2_name] = pd.to_numeric(df_result[new_col2_name], errors='coerce')
            
            # Calculate ratio
            ratio_col_name = f"{new_col1_name.split(' (')[0]}/{new_col2_name.split(' (')[0]} Ratio"
            df_result[ratio_col_name] = df_result[new_col1_name] / df_result[new_col2_name]
            
            print(f"Successfully split '{column_to_split}' into:")
            print(f"  - {new_col1_name}")
            print(f"  - {new_col2_name}")
            print(f"  - {ratio_col_name}")
        else:
            print(f"Warning: Could not split '{column_to_split}' - unexpected format")
    else:
        print(f"Warning: Column '{column_to_split}' not found in DataFrame")
    
    return df_result

In [136]:
#Split labor force partition and calculate the %
df_economic_indicators_flat = split_ratio_column(df_economic_indicators_flat,"Labour force participation rate (female/male pop. %)",female_col, male_col)
df_economic_indicators_flat.head()

Successfully split 'Labour force participation rate (female/male pop. %)' into:
  - Labour force participation rate - Female (%)
  - Labour force participation rate - Male (%)
  - Labour force participation rate - Female/Labour force participation rate - Male Ratio


,Country,Year,Agricultural production index (2014-2016=100),"Balance of payments, current account (million US$)",CPI: Consumer Price Index (2010=100),Economy: Agriculture (% of Gross Value Added),Economy: Industry (% of Gross Value Added),Economy: Services and other activity (% of GVA),Employment in agriculture (% of employed),Employment in industry (% of employed),Employment in services (% employed),"GDP growth rate (annual %, const. 2015 prices)",GDP per capita (current US$),GDP: Gross domestic product (million current US$),International trade: balance (million current US$),International trade: exports (million current US$),International trade: imports (million current US$),Labour force participation rate (female/male pop. %),Unemployment (% of labour force),Labour force participation rate - Female (%),Labour force participation rate - Male (%),Labour force participation rate - Female/Labour force participation rate - Male Ratio
0,Afghanistan,2015.0,96.0,4193.0,133.0,22.0,NaN,68.3,44.6,20.7,NaN,2.5,554.0,18700.0,7151.0,571.0,7723.0,19.1 / 75.3i,9.0,19.1,75.3,0.253652
1,Afghanistan,2020.0,112.0,3137.0,150.0,31.4,NaN,55.0,46.0,18.5,NaN,2.1,512.7,19983.0,7484.0,783.0,8266.0,16.5 / 66.7,11.7,16.5,66.7,0.247376
2,Afghanistan,2024.0,111.0,NaN,NaN,35.5,NaN,47.5,46.6,18.3,NaN,6.2,344.6,14175.0,4337.0,456.0,4793.0,5.0 / 69.3i,16.3,5.0,69.3,0.072150
3,Albania,2015.0,100.0,980.0,111.0,22.5,24.8,52.7,41.3,18.7,NaN,2.2,3950.4,11387.0,2391.0,NaN,NaN,47.0 / 64.1,17.2,47.0,64.1,0.733229
4,Albania,2020.0,106.0,1315.0,121.0,21.9,22.9,55.1,36.2,20.6,NaN,3.5,5289.0,15163.0,2995.0,NaN,NaN,50.1 / 64.7i,12.8,50.1,64.7,0.774343


In [137]:
df_economic_indicators_flat = df_economic_indicators_flat.drop(columns=["Labour force participation rate (female/male pop. %)"],axis=1)
df_economic_indicators_flat.head()

,Country,Year,Agricultural production index (2014-2016=100),"Balance of payments, current account (million US$)",CPI: Consumer Price Index (2010=100),Economy: Agriculture (% of Gross Value Added),Economy: Industry (% of Gross Value Added),Economy: Services and other activity (% of GVA),Employment in agriculture (% of employed),Employment in industry (% of employed),Employment in services (% employed),"GDP growth rate (annual %, const. 2015 prices)",GDP per capita (current US$),GDP: Gross domestic product (million current US$),International trade: balance (million current US$),International trade: exports (million current US$),International trade: imports (million current US$),Unemployment (% of labour force),Labour force participation rate - Female (%),Labour force participation rate - Male (%),Labour force participation rate - Female/Labour force participation rate - Male Ratio
0,Afghanistan,2015.0,96.0,4193.0,133.0,22.0,NaN,68.3,44.6,20.7,NaN,2.5,554.0,18700.0,7151.0,571.0,7723.0,9.0,19.1,75.3,0.253652
1,Afghanistan,2020.0,112.0,3137.0,150.0,31.4,NaN,55.0,46.0,18.5,NaN,2.1,512.7,19983.0,7484.0,783.0,8266.0,11.7,16.5,66.7,0.247376
2,Afghanistan,2024.0,111.0,NaN,NaN,35.5,NaN,47.5,46.6,18.3,NaN,6.2,344.6,14175.0,4337.0,456.0,4793.0,16.3,5.0,69.3,0.072150
3,Albania,2015.0,100.0,980.0,111.0,22.5,24.8,52.7,41.3,18.7,NaN,2.2,3950.4,11387.0,2391.0,NaN,NaN,17.2,47.0,64.1,0.733229
4,Albania,2020.0,106.0,1315.0,121.0,21.9,22.9,55.1,36.2,20.6,NaN,3.5,5289.0,15163.0,2995.0,NaN,NaN,12.8,50.1,64.7,0.774343


## Clean Social Indicators 

In [138]:
df_social_indicators.head()

,Country,Category,Indicator,Year,Value
61,Afghanistan,Social indicators,Population growth rate (average annual %),2015.0,2.3
62,Afghanistan,Social indicators,Population growth rate (average annual %),2020.0,3.3
63,Afghanistan,Social indicators,Population growth rate (average annual %),2024.0,2.8a
64,Afghanistan,Social indicators,Urban population (% of total population),2015.0,24.8
65,Afghanistan,Social indicators,Urban population (% of total population),2020.0,25.8c


In [139]:
df_social_indicators = df_social_indicators.drop(columns=["Category"],axis=1)
df_social_indicators.head()

,Country,Indicator,Year,Value
61,Afghanistan,Population growth rate (average annual %),2015.0,2.3
62,Afghanistan,Population growth rate (average annual %),2020.0,3.3
63,Afghanistan,Population growth rate (average annual %),2024.0,2.8a
64,Afghanistan,Urban population (% of total population),2015.0,24.8
65,Afghanistan,Urban population (% of total population),2020.0,25.8c


In [140]:
df_social_indicators_flat = pivot_df(df_social_indicators, idx_cols=["Country","Year"])
df_social_indicators_flat.head()

,Country,Year,Education: Government expenditure (% of GDP),Education: Government expendituref (% of GDP),Education: Government expenditurei (% of GDP),Education: Lowr. sec. gross enrol. ratio (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio f (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio g (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio i (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio j (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio k (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio l (f/m per 100 pop.),Education: Lowr. sec. gross enrol. ratio o (f/m per 100 pop.),Education: Primary gross enrol. ratio (f/m per 100 pop.),Education: Primary gross enrol. ratiof (f/m per 100 pop.),Education: Primary gross enrol. ratiog (f/m per 100 pop.),Education: Primary gross enrol. ratioi (f/m per 100 pop.),Education: Primary gross enrol. ratioj (f/m per 100 pop.),Education: Primary gross enrol. ratiok (f/m per 100 pop.),Education: Primary gross enrol. ratiol (f/m per 100 pop.),Education: Primary gross enrol. ratioo (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratio (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratiof (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratiog (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratioi (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratioj (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratiok (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratiol (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratioo (f/m per 100 pop.),"Fertility rate, total (live births per woman)","Fertility rate, totala (live births per woman)","Fertility rate, totalb (live births per woman)","Fertility rate, totald (live births per woman)","Fertility rate, totalk (live births per woman)","Fertility rate, totalo (live births per woman)",Health: Current expenditure (% of GDP),"Health: Current expenditurec,o (% of GDP)","Health: Current expendituree,i (% of GDP)","Health: Current expendituree,j (% of GDP)","Health: Current expendituref,j (% of GDP)","Health: Current expendituref,j,k (% of GDP)","Health: Current expendituref,m,n (% of GDP)","Health: Current expendituref,o,p (% of GDP)","Health: Current expendituref,s (% of GDP)","Health: Current expenditureg,j (% of GDP)","Health: Current expenditureg,l (% of GDP)",Health: Current expenditurei (% of GDP),"Health: Current expenditurei,j,k (% of GDP)","Health: Current expenditurei,m (% of GDP)","Health: Current expenditurei,o (% of GDP)","Health: Current expenditurei,o,p (% of GDP)","Health: Current expenditurei,t,u (% of GDP)",Health: Current expenditurej (% of GDP),"Health: Current expenditurej,k (% of GDP)","Health: Current expenditurej,k,l (% of GDP)","Health: Current expenditurej,o,p (% of GDP)","Health: Current expenditurej,p (% of GDP)","Health: Current expenditurej,p,q (% of GDP)","Health: Current expenditurej,q (% of GDP)","Health: Current expenditurej,r (% of GDP)",Health: Current expenditurek (% of GDP),"Health: Current expenditurek,l (% of GDP)","Health: Current expenditurek,q (% of GDP)","Health: Current expenditurek,q,r (% of GDP)",Health: Current expenditurel (% of GDP),"Health: Current expenditurel,m (% of GDP)",Health: Current expenditurem (% of GDP),"Health: Current expenditurem,n (% of GDP)",Health: Current expendituren (% of GDP),"Health: Current expendituren,o (% of GDP)",Health: Current expenditureo (% of GDP),"Health: Current expenditureo,p (% of GDP)","Health: Current expenditureo,p,q (% of GDP)","Health: Current expenditureo,q (% of GDP)",Health: Current expenditurep (% of GDP),"Health: Current expenditurep,q (% of GDP)",Health: Current expenditureq (% of GDP),"Health: Current expenditureq,r (% of GDP)",Health: Current expenditurer (% of GDP),"Health: Current expenditurer,s (% of GDP)",Health: Current expenditures (% of GDP),Health: Current expendituret (% of GDP),Health: Current expenditureu (% of GDP),Health: Physicians (per 1 000 pop.),Intenti

In [141]:
print(list(df_social_indicators_flat.columns))

['Country', 'Year', 'Education: Government expenditure (% of GDP)', 'Education: Government expendituref (% of GDP)', 'Education: Government expenditurei (% of GDP)', 'Education: Lowr. sec. gross enrol. ratio  (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio f (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio g (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio i (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio j (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio k (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio l (f/m per 100 pop.)', 'Education: Lowr. sec. gross enrol. ratio o (f/m per 100 pop.)', 'Education: Primary gross enrol. ratio (f/m per 100 pop.)', 'Education: Primary gross enrol. ratiof (f/m per 100 pop.)', 'Education: Primary gross enrol. ratiog (f/m per 100 pop.)', 'Education: Primary gross enrol. ratioi (f/m per 100 pop.)', 'Education: Primary gross enrol. ratioj (f/m per 100 pop.)', 'Education:

In [142]:
root_column_names = [
    'Country',
    'Year',
    'Education: Government expenditure (% of GDP)',
    'Education: Lowr. sec. gross enrol. ratio (f/m per 100 pop.)',
    'Education: Primary gross enrol. ratio (f/m per 100 pop.)',
    'Education: Upr. Sec. gross enrol. ratio (f/m per 100 pop.)',
    'Fertility rate, total (live births per woman)',
    'Health: Current expenditure (% of GDP)',
    'Health: Physicians (per 1 000 pop.)',
    'Intentional homicide rate (per 100 000 pop.)',
    'International migrant stock (000/% of total pop.)',
    'Life expectancy at birth (females/males, years)',
    'Population age distributiona,h (0-14/60+ years old, %)',
    'Population growth rate (average annual %)',
    'Refugees and others of concern to UNHCR (000)',
    'Seats held by women in national parliaments (%)',
    'Under five mortality rate (per 1000 live births)',
    'Urban population (% of total population)',
    'Urban population growth rate (average annual %)'
]

df_social_indicators_flat = smart_combine_typo_columns(df_social_indicators_flat, root_column_names)
print(f"\nFinal shape: {df_social_indicators_flat.shape}")
print(f"Final columns: {list(df_social_indicators_flat.columns)}")
df_economic_indicators_flat = df_economic_indicators_flat.rename(columns={"Population age distributiona,h (0-14/60+ years old, %)":"Population age distribution (0-14/60+ years old, %)"})
df_social_indicators_flat.head()


Merging into 'Education: Government expenditure (% of GDP)':
  Found 2 similar columns
  Merging: 'Education: Government expendituref (% of GDP)' -> 'Education: Government expenditure (% of GDP)' (similarity: 0.989)
  Merging: 'Education: Government expenditurei (% of GDP)' -> 'Education: Government expenditure (% of GDP)' (similarity: 0.989)

Merging into 'Education: Primary gross enrol. ratio (f/m per 100 pop.)':
  Found 22 similar columns
  Merging: 'Education: Lowr. sec. gross enrol. ratio  (f/m per 100 pop.)' -> 'Education: Primary gross enrol. ratio (f/m per 100 pop.)' (similarity: 0.862)
  Merging: 'Education: Lowr. sec. gross enrol. ratio f (f/m per 100 pop.)' -> 'Education: Primary gross enrol. ratio (f/m per 100 pop.)' (similarity: 0.855)
  Merging: 'Education: Lowr. sec. gross enrol. ratio g (f/m per 100 pop.)' -> 'Education: Primary gross enrol. ratio (f/m per 100 pop.)' (similarity: 0.855)
  Merging: 'Education: Lowr. sec. gross enrol. ratio i (f/m per 100 pop.)' -> 'Educ

,Country,Year,Education: Government expenditure (% of GDP),Education: Primary gross enrol. ratio (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratio (f/m per 100 pop.),"Fertility rate, total (live births per woman)",Health: Current expenditure (% of GDP),Health: Physicians (per 1 000 pop.),Intentional homicide rate (per 100 000 pop.),International migrant stock (000/% of total pop.),"Life expectancy at birth (females/males, years)","Population age distributiona,h (0-14/60+ years old, %)",Population growth rate (average annual %),Refugees and others of concern to UNHCR (000),Seats held by women in national parliaments (%),Under five mortality rate (per 1000 live births),Urban population (% of total population),Urban population growth rate (average annual %)
0,Afghanistan,2015.0,3.3,85.9 / 125.4,27.9 / 54.3,5.7,10.1,0.3,10,339.4 / 1.0m,64.2 / 60.4,46.3 / 3.7,2.3,1 421.4,27.7,76.5,24.8,4k
1,Afghanistan,2020.0,2.9,88.3 / 130.6c,29.3 / 54.1q,5.1,15.5o,0.3,6.6,144.1 / 0.4m,64.7 / 58.5,44.2 / 3.8,3.3,2 802.9,27r,62.9,25.8c,...
2,Afghanistan,2024.0,...,... / ...,... / ...,4.8a,"21.8o,p",...,4p,... / ...,67.8 / 64.7a,42.9 / 3.9a,2.8a,3 346.2j,"27r,s,b",54.5a,...,...
3,Albania,2015.0,3.4,111.5 / 115.0,91.5 / 105.3,1.6,6.5,1.3o,2.2,52.0 / 1.8,80.5 / 76.3,19.0 / 17.7,-0.2,8.1,20.7,10.1,57.4,1.8l
4,Albania,2020.0,3.3,98.0 / 103.2,89.4 / 97.2,1.4,7.5,1.9,2.1,48.8 / 1.7,80.3 / 75.4,17.5 / 21.1,-0.4,3.6,29.5p,9.4,61.2c,...


In [143]:
numerical_cols=[
    "Education: Government expenditure (% of GDP)",
    "Fertility rate, total (live births per woman)",
    "Health: Current expenditure (% of GDP)",
    "Health: Physicians (per 1 000 pop.)",
    "Intentional homicide rate (per 100 000 pop.)",
    "Population growth rate (average annual %)",
    "Refugees and others of concern to UNHCR (000)",
    "Seats held by women in national parliaments (%)",
    "Under five mortality rate (per 1000 live births)",
    "Urban population (% of total population)",
    "Urban population growth rate (average annual %)"
]
df_social_indicators_flat = clean_numerical_cols(df_social_indicators_flat, numerical_cols)
df_social_indicators_flat.head()

,Country,Year,Education: Government expenditure (% of GDP),Education: Primary gross enrol. ratio (f/m per 100 pop.),Education: Upr. Sec. gross enrol. ratio (f/m per 100 pop.),"Fertility rate, total (live births per woman)",Health: Current expenditure (% of GDP),Health: Physicians (per 1 000 pop.),Intentional homicide rate (per 100 000 pop.),International migrant stock (000/% of total pop.),"Life expectancy at birth (females/males, years)","Population age distributiona,h (0-14/60+ years old, %)",Population growth rate (average annual %),Refugees and others of concern to UNHCR (000),Seats held by women in national parliaments (%),Under five mortality rate (per 1000 live births),Urban population (% of total population),Urban population growth rate (average annual %)
0,Afghanistan,2015.0,3.3,85.9 / 125.4,27.9 / 54.3,5.7,10.1,0.3,10.0,339.4 / 1.0m,64.2 / 60.4,46.3 / 3.7,2.3,1421.4,27.7,76.5,24.8,4.0
1,Afghanistan,2020.0,2.9,88.3 / 130.6c,29.3 / 54.1q,5.1,15.5,0.3,6.6,144.1 / 0.4m,64.7 / 58.5,44.2 / 3.8,3.3,2802.9,27.0,62.9,25.8,NaN
2,Afghanistan,2024.0,NaN,... / ...,... / ...,4.8,21.8,NaN,4.0,... / ...,67.8 / 64.7a,42.9 / 3.9a,2.8,3346.2,27.0,54.5,NaN,NaN
3,Albania,2015.0,3.4,111.5 / 115.0,91.5 / 105.3,1.6,6.5,1.3,2.2,52.0 / 1.8,80.5 / 76.3,19.0 / 17.7,0.2,8.1,20.7,10.1,57.4,1.8
4,Albania,2020.0,3.3,98.0 / 103.2,89.4 / 97.2,1.4,7.5,1.9,2.1,48.8 / 1.7,80.3 / 75.4,17.5 / 21.1,0.4,3.6,29.5,9.4,61.2,NaN


In [ ]:
# Define columns to split and their new column names
columns_to_split = [
    {
        "column": "Education: Primary gross enrol. ratio (f/m per 100 pop.)",
        "col1": "Education: Primary gross enrol. ratio - Female (per 100 pop.)",
        "col2": "Education: Primary gross enrol. ratio - Male (per 100 pop.)"
    },
    {
        "column": "Education: Upr. Sec. gross enrol. ratio (f/m per 100 pop.)",
        "col1": "Education: Upper Sec. gross enrol. ratio - Female (per 100 pop.)",
        "col2": "Education: Upper Sec. gross enrol. ratio - Male (per 100 pop.)"
    },
    {
        "column": "Education: Lowr. sec. gross enrol. ratio (f/m per 100 pop.)",
        "col1": "Education: Lower Sec. gross enrol. ratio - Female (per 100 pop.)",
        "col2": "Education: Lower Sec. gross enrol. ratio - Male (per 100 pop.)"
    },
    {
        "column": "International migrant stock (000/% of total pop.)",
        "col1": "International migrant stock (000)",
        "col2": "International migrant stock (% of total pop.)"
    },
    {
        "column": "Life expectancy at birth (females/males, years)",
        "col1": "Life expectancy at birth - Female (years)",
        "col2": "Life expectancy at birth - Male (years)"
    },
    {
        "column": "Population age distribution (0-14/60+ years old, %)",
        "col1": "Population age distribution - 0-14 years (%)",
        "col2": "Population age distribution - 60+ years (%)"
    }
]

# Apply the split function to all columns
for split_info in columns_to_split:
    if split_info["column"] in df_social_indicators_flat.columns:
        print(f"\nProcessing: {split_info['column']}")
        df_social_indicators_flat = split_ratio_column(
            df=df_social_indicators_flat,
            column_to_split=split_info["column"],
            new_col1_name=split_info["col1"],
            new_col2_name=split_info["col2"]
        )
        # Drop the original column after splitting
        df_social_indicators_flat = df_social_indicators_flat.drop(columns=[split_info["column"]], axis=1)
    else:
        print(f"Warning: Column '{split_info['column']}' not found in dataframe")

print(f"\nFinal shape after splitting: {df_social_indicators_flat.shape}")
df_social_indicators_flat.head()